In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import time

# -------------------------------------
# 0) device 설정
# -------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------
# 1) 모델 선언 및 마지막 레이어 교체
# -------------------------------------
model = models.efficientnet_b0(pretrained=True)  # ImageNet 사전학습
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 3)  # 3-class (AIGenerated, Fake, Real)

# -------------------------------------
# 2) 데이터 로더 설정
# -------------------------------------
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os

base_dir = os.getcwd()  # 현재 작업 디렉토리
train_dir = os.path.join(base_dir, 'Train')
val_dir   = os.path.join(base_dir, 'Validation')
test_dir  = os.path.join(base_dir, 'Test')

batch = 16
img_resized = (128, 128)

train_transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=test_transform)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=test_transform)

train_loader  = DataLoader(train_dataset, batch_size=batch, shuffle=True,  num_workers=0, pin_memory=True)
val_loader    = DataLoader(val_dataset,   batch_size=batch, shuffle=False, num_workers=0, pin_memory=True)
test_loader   = DataLoader(test_dataset,  batch_size=batch, shuffle=False, num_workers=0, pin_memory=True)

# 클래스 인덱스 확인
print("Train Dataset class to index mapping:")
print(train_dataset.class_to_idx)


In [ ]:

# -------------------------------------
# 3) 학습 설정 (Loss, Optimizer 등)
# -------------------------------------
import torch.optim as optim

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
from tqdm import tqdm

# -------------------------------------
# 4) Training Loop
# -------------------------------------
total_start_time = time.time()

for epoch in range(num_epochs):
    start_time = time.time()  # 에포크 시작 시간 기록
    
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc  = correct / total

    # validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item() * val_images.size(0)
            _, v_preds = torch.max(val_outputs, 1)
            val_correct += torch.sum(v_preds == val_labels).item()
            val_total += val_labels.size(0)

    val_epoch_loss = val_loss / val_total
    val_epoch_acc  = val_correct / val_total
    
    end_time = time.time()  # 에포크 종료 시간 기록
    epoch_duration = end_time - start_time  # 에포크 소요 시간 계산

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {epoch_loss:.4f} Train Acc: {epoch_acc:.4f} | "
          f"Val Loss: {val_epoch_loss:.4f} Val Acc: {val_epoch_acc:.4f} | "
          f"Time: {epoch_duration:.2f} sec")

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total Training Time: {total_duration/60:.2f} minutes")

# -------------------------------------
# 5) Model Save (pth)
# -------------------------------------
pth_name = "binary_test_3class_128.pth"
torch.save(model.state_dict(), pth_name)


In [ ]:

# -------------------------------------
# 6) Test
# -------------------------------------
model.eval()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

avg_loss = test_loss / total
accuracy = correct / total

print(f"[Test] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


In [9]:
# --------------------------------------------------
# (추가) 학습된 모델을 ONNX 포맷으로 내보내기
# --------------------------------------------------
import torch
import torchvision.models as models
import torch.nn as nn
import torch.onnx

pth_name = "face_3class_s128_e10.pth"

# 1) 모델 구조 생성 & 파라미터 로드
onnx_model = models.efficientnet_b0(pretrained=False)
num_features = onnx_model.classifier[1].in_features
onnx_model.classifier[1] = nn.Linear(num_features, 3)
onnx_model.load_state_dict(torch.load(pth_name, map_location="cpu"))
onnx_model.eval()

# 2) ONNX 변환
onnx_model_name = "efficientnet_b0_3class_128.onnx"

# (예시) 1개의 이미지(batch_size=1), 3채널, 128x128 크기
dummy_input = torch.randn(1, 3, 128, 128, device='cpu')

torch.onnx.export(
    onnx_model,
    dummy_input,
    onnx_model_name,
    input_names=["input"],        # ONNX 모델에 들어갈 입력 이름
    output_names=["output"],      # ONNX 모델에서 뽑을 출력 이름
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=11,             # ONNX Opset 버전 (필요에 따라 변경)
    do_constant_folding=True      # 상수 폴딩 최적화
)

print(f"ONNX 모델 저장 완료: {onnx_model_name}")


C:\Users\user\AppData\Local\Temp\ipykernel_20408\2688192957.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  onnx_model.load_state_dict(torch.load(pth_name, map_location

ONNX 모델 저장 완료: efficientnet_b0_3class_128.onnx


In [7]:
! pip install onnx

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   --------------------------- ------------ 10.0/14.5 MB 51.6 MB/s eta 0:00:01
   ---------------------------------------- 14.5/14.5 MB 48.0 MB/s eta 0:00:00


In [10]:
! pip install --upgrade onnxruntime-gpu

   ---------------------------------------- 0.0/279.7 MB ? eta -:--:--
   - -------------------------------------- 10.2/279.7 MB 53.0 MB/s eta 0:00:06
   -- ------------------------------------- 15.7/279.7 MB 52.1 MB/s eta 0:00:06
   --- ------------------------------------ 21.2/279.7 MB 33.6 MB/s eta 0:00:08
   ---- ----------------------------------- 31.5/279.7 MB 39.1 MB/s eta 0:00:07
   ---- ----------------------------------- 32.2/279.7 MB 31.5 MB/s eta 0:00:08
   ----- ---------------------------------- 37.0/279.7 MB 29.7 MB/s eta 0:00:09
   ------ --------------------------------- 48.2/279.7 MB 33.0 MB/s eta 0:00:08
   -------- ------------------------------- 57.7/279.7 MB 35.7 MB/s eta 0:00:07
   --------- ------------------------------ 64.0/279.7 MB 34.0 MB/s eta 0:00:07
   ----------- ---------------------------- 77.1/279.7 MB 36.9 MB/s eta 0:00:06
   ------------ --------------------------- 84.7/279.7 MB 37.0 MB/s eta 0:00:06
   -------------- ------------------------- 98.0/